In [2]:
# import packages
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker as mtick
from scipy import special

# switch on inline plotting
%matplotlib inline

# append path for loading own packages
import sys
sys.path.append("../packages")

# import own packages
import edges
import grating
import copy
from auxiliary import theta
import auxiliary as aux
import jitter
import density
import scattering

In [3]:
def present_density_and_photons(a, s, **kwargs):
    xmin    = kwargs.pop('xmin', -1e-6)
    xmax    = kwargs.pop('xmax',  1e-6)
    ymin    = kwargs.pop('ymin', -1e-6)
    ymax    = kwargs.pop('ymax',  1e-6)
    
    dx      = (xmax-xmin)/(a.shape[0]-1)
    dy      = (ymax-ymin)/(a.shape[1]-1)
    
    qxmin   = kwargs.pop('qxmin', -np.pi/dx)
    qxmax   = kwargs.pop('qxmax',  np.pi/dx)
    qymin   = kwargs.pop('qymin', -np.pi/dy)
    qymax   = kwargs.pop('qymax',  np.pi/dy)
    qy      = np.linspace(qymin*10e-9, qymax*10e-9, s.shape[0])
    
    comment = kwargs.pop('comment', '')
    
    
    fig     = plt.figure(figsize=(20, 30))
    
    sub1 = fig.add_subplot(3, 2, 1)
    aux.imshow(a.T, extent = [ymin/10e-6, ymax/10e-6, xmin/10e-6, xmax/10e-6])
    #sub1.get_xaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
    #sub1.get_yaxis().set_major_formatter(mtick.FormatStrFormatter('%.1e'))
    sub1.set_xlabel('y (laser coordinates) [mu]', fontsize=18)
    sub1.set_ylabel('x (laser coordinates) [mu]', fontsize=18)
    plt.colorbar()
    
    sub2 = fig.add_subplot(3, 2, 2)
    aux.imshow(np.log(s+1).T, extent = [qymin*10e-9, qymax*10e-9, qxmin*10e-9, qxmax*10e-9])
    sub2.set_xlabel('q_y (laser coordinates) [1/nm]', fontsize=18)
    sub2.set_ylabel('q_x (laser coordinates) [1/nm]', fontsize=18)
    plt.colorbar()
        
    sub3 = fig.add_subplot(3, 2, 3)
    plt.plot(qy, s[:, s.shape[1]//2], label='N(photons)')
    plt.legend()
    plt.title('Line-out of photon image along q_y at q_x = 0', fontsize=22)
    sub3.set_xlabel('q_y (laser coordinates) [1/nm]', fontsize=18)
    
    sub4 = fig.add_subplot(3, 2, 4)
    plt.plot(qy, np.log(s[:, s.shape[1]//2]+1), label='log(N(photons)+1)')
    plt.legend()
    plt.title('Line-out of photon image along q_y at q_x = 0\nlog scale', fontsize=22)
    sub4.set_xlabel('q_y (laser coordinates) [1/nm]', fontsize=18)
    
    sub5 = fig.add_subplot(3, 2, 5)
    plt.axis('off')
    sub5.text(0.05, 0.95, comment,
        verticalalignment='top', horizontalalignment='left',
        transform=sub5.transAxes, fontsize=18)
    
    return fig
    

In [10]:
# Load densities

dirpath             = "../results/Scan5_Test/"
density_fns_list_fn  = "density_filenames.txt"

density_fns         = [dirpath+line.rstrip('\n') for line in open(dirpath+density_fns_list_fn)]

dfns = density_fns
densities = {}
for dfn in dfns:
    densities[dfn] = np.loadtxt(dfn)
    print(dfn)

../results/Scan5_Test/020perfedge_400pitch_015jampl_050jwl_000sigma_only_foil_summeddensity.txt
../results/Scan5_Test/020perfedge_400pitch_015jampl_050jwl_020sigma_both_summeddensity.txt
../results/Scan5_Test/020perfedge_400pitch_015jampl_050jwl_020sigma_only_feat_summeddensity.txt
../results/Scan5_Test/020perfedge_400pitch_015jampl_050jwl_020sigma_only_foil_summeddensity.txt


In [11]:
only_foil_sharp  = densities[density_fns[0]] * 5.95e3
both             = densities[density_fns[1]] * 5.95e3
only_feat        = densities[density_fns[2]] * 5.95e3
only_foil_smooth = densities[density_fns[3]] * 5.95e3

In [14]:
d = [(both,                              'both'),\
     (only_feat+only_foil_sharp,         'only_feat+only_foil_sharp'),\
     (only_feat+only_foil_smooth,        'only_feat+only_foil_smooth'),\
     (both-(only_feat+only_foil_sharp),  'both-(only_feat+only_foil_sharp)'),\
     (both-(only_feat+only_foil_smooth), 'both-(only_feat+only_foil_smooth)')
    ]

for i in d:
    a = i[0]
    s = scattering.propagate(a, 1e10, 0.155e-9, 2e-6, 1, 1)
    
    kwargs = {}
    kwargs['comment'] =\
        ''
    fig = present_density_and_photons(a.T, s.T, **kwargs)
    fig.suptitle(i[1], fontsize=24, fontweight='bold')
    
    path = dirpath + i[1] + '_presentation.pdf'
    if not aux.path_exists(path):
        fig.savefig(path)
    plt.close(fig)


In [129]:
# Scatter and present

for dfn in dfns:
    # pitch: 50 nm
    #deltaz = 2.035
    #dV     = 7.58
    #dn     = 5.31
    
    # pitch: 100 nm
    #deltaz = 2.071
    #dV     = 7.72
    #dn     = 5.40
    
    # pitch: 200 nm
    #deltaz = 2.141
    #dV     = 7.98
    #dn     = 5.59
    
    # pitch: 400 nm
    deltaz = 2.282
    dV     = 8.50
    dn     = 5.95
    
    a = densities[dfn] * dn * 1e3
    s = scattering.propagate(a, 1e10, 0.155e-9, 2e-6, 1, 1)
    
    kwargs = {}
    kwargs['comment'] =\
        'X-Ray: I_total = 1e10 photons, lambda = 0.155nm\n\n\
Normalization factor for density: Number of electrons per voxel for silicon\n\
Focus size = (2 mu)**2;   focus resolution = (1024)**2\n\
distance between density slices = {} mu / 1024 (value depends on pitch)\n\
=> voxel volume = {} (nm)**3    => #electrons per voxel for silicon = {}e3'.format(deltaz, dV, dn)
    fig = present_density_and_photons(a.T, s.T, **kwargs)
    fig.suptitle(dfn, fontsize=24, fontweight='bold')
    
    path = dfn + '_presentation.pdf'
    if not aux.path_exists(path):
        fig.savefig(path)
    plt.close(fig)
    